# Amazon Bedrock Guardrails Enforcements - End-to-End Tutorial

## Overview

This notebook demonstrates how to implement **Amazon Bedrock Guardrails Enforcements**, a powerful feature that enables centralized safety controls across AWS accounts and organizations.

### What You'll Learn

- How to create guardrails with supported filters for enforcement
- How to implement organization-level guardrail enforcement
- How to implement account-level guardrail enforcement
- How to test and verify guardrail enforcement
- How to implement layered guardrails for defense in depth

### Key Concepts

**Organization-level enforcement**: Apply guardrails across OUs, individual accounts, or entire organizations using AWS Organizations policies

**Account-level enforcement**: Designate a guardrail version within an AWS account for all Bedrock model invocations

**Layered protection**: Combine organization, account, and application-specific guardrails with union of controls (most restrictive takes precedence)

### Prerequisites

- AWS account with appropriate IAM permissions
- For organization-level: AWS Organizations management account access
- For account-level: Account administrator access
- Python 3.8 or later
- boto3 library installed

---

## 1. Setup and Installation

First, let's install the required dependencies and initialize our AWS clients.

In [ ]:
# Install required packages
!pip install boto3 awscli --upgrade

In [ ]:
import boto3
import json
import time
from typing import Dict, List, Optional
from botocore.exceptions import ClientError

# Initialize AWS clients
REGION = 'us-west-2'  # Change to your preferred region

session = boto3.Session(profile_name='default')
bedrock_client = session.client('bedrock', region_name=REGION)
bedrock_runtime_client = session.client('bedrock-runtime', region_name=REGION)
organizations_client = session.client('organizations')
iam_client = session.client('iam')
sts_client = session.client('sts')

# Get current account information
account_id = sts_client.get_caller_identity()['Account']
print(f"✓ Initialized clients in region: {REGION}")
print(f"✓ Current AWS Account ID: {account_id}")

### Important API Information

This tutorial uses the following Bedrock Guardrails Enforcements APIs:

**For Resource-Based Policies:**
- `put_resource_policy()` - Attach a resource-based policy to a guardrail for cross-account access

**For Account-Level Enforcement:**
- `put_enforced_guardrail_configuration()` - Enable account-level guardrail enforcement
- `list_enforced_guardrail_configurations()` - List all enforced guardrail configurations in an account
- `delete_enforced_guardrail_configuration()` - Remove account-level enforcement

**For Organization-Level Enforcement:**
- AWS Organizations APIs (`enable_policy_type`, `create_policy`, `attach_policy`)

**For Testing:**
- `apply_guardrail()` - Test guardrail behavior directly
- `invoke_model()` / `converse()` - Test with actual model invocations

---


---

## Part 1: Creating Guardrails for Enforcement

### Supported Guardrail Filters

When creating guardrails for enforcement, the following filters are supported:
- **Content filters**: Block harmful content (sexual, violence, hate, insults, misconduct)
- **Denied topics**: Prevent responses on specific topics
- **Word filters**: Block specific words or use managed word lists
- **Sensitive information filters**: Detect and block PII
- **Contextual grounding checks**: Verify responses are grounded in source material

⚠️ **IMPORTANT**: Do NOT include the automated reasoning policy - it is unsupported for guardrail enforcements and will cause runtime failures.

### Step 1.1: Create a Guardrail

Let's create a guardrail with multiple safety filters.

In [ ]:
def create_guardrail(name: str, description: str) -> Dict:
    """
    Create a guardrail with supported filters for enforcement.
    """
    print(f"Creating Guardrail: {name}")
    print("="*70)
    
    try:
        # Configure content filters (block harmful content)
        content_policy_config = {
            'filtersConfig': [
                {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
                {'type': 'HATE', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'},
                {'type': 'INSULTS', 'inputStrength': 'MEDIUM', 'outputStrength': 'MEDIUM'}
            ],
            'tierConfig': {
                'tierName': 'STANDARD'
            }
        }
        
        # Configure denied topics (example: financial advice)
        topic_policy_config = {
            'topicsConfig': [
                {
                    'name': 'Financial Advice',
                    'definition': 'Providing specific investment, trading, or financial planning advice',
                    'examples': [
                        'Should I invest in this stock?',
                        'What cryptocurrency should I buy?'
                    ],
                    'type': 'DENY'
                }
            ],
            'tierConfig': {
                'tierName': 'STANDARD'
            }
        }
        
        # Configure word filters (block profanity)
        word_policy_config = {
            'wordsConfig': [{'text': 'badword1'}, {'text': 'badword2'}],
            'managedWordListsConfig': [{'type': 'PROFANITY'}]
        }
        
        # Configure sensitive information filters (PII)
        sensitive_info_policy_config = {
            'piiEntitiesConfig': [
                {'type': 'EMAIL', 'action': 'BLOCK'},
                {'type': 'PHONE', 'action': 'BLOCK'},
                {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
            ]
        }
        
        response = bedrock_client.create_guardrail(
            name=name,
            description=description,
            contentPolicyConfig=content_policy_config,
            topicPolicyConfig=topic_policy_config,
            wordPolicyConfig=word_policy_config,
            sensitiveInformationPolicyConfig=sensitive_info_policy_config,
            blockedInputMessaging="I cannot process this request due to safety policies.",
            blockedOutputsMessaging="I cannot provide this response due to safety policies.",
            crossRegionConfig={
                'guardrailProfileIdentifier': 'us.guardrail.v1:0'
            },
        )
        
        guardrail_id = response['guardrailId']
        guardrail_arn = response['guardrailArn']
        
        print(f"✓ Guardrail created successfully!")
        print(f"  Guardrail ID: {guardrail_id}")
        print(f"  Guardrail ARN: {guardrail_arn}")
        
        return {
            'guardrailId': guardrail_id,
            'guardrailArn': guardrail_arn,
            'version': response.get('version', 'DRAFT')
        }
        
    except ClientError as e:
        print(f"✗ Error creating guardrail: {e}")
        raise

# Create the guardrail
guardrail = create_guardrail(
    name="EnforcementDemoGuardrail3",
    description="Demonstration guardrail for enforcement tutorial"
)

### Step 1.2: Create a Guardrail Version

Versions are **required** for enforcement to ensure guardrails cannot be modified after deployment. Creating a version makes the guardrail immutable.

In [ ]:
def create_guardrail_version(guardrail_id: str, description: str = "") -> Dict:
    """
    Create an immutable version of the guardrail.
    """
    print(f"\nCreating Guardrail Version")
    print("="*70)
    
    try:
        response = bedrock_client.create_guardrail_version(
            guardrailIdentifier=guardrail_id,
            description=description or f"Version created at {time.strftime('%Y-%m-%d %H:%M:%S')}"
        )
        
        version = response['version']
        
        print(f"✓ Guardrail version created successfully!")
        print(f"  Version: {version}")
        
        # Wait for version to be ready
        print("  Waiting for version to be ready...")
        time.sleep(5)
        
        return {'version': version, 'guardrailId': guardrail_id}
        
    except ClientError as e:
        print(f"✗ Error creating guardrail version: {e}")
        raise

# Create version
version_info = create_guardrail_version(
    guardrail_id=guardrail['guardrailId'],
    description="Production version for enforcement"
)

### Step 1.3: Attach Resource-Based Policy

Resource-based policies are:
- **REQUIRED** for organization-level enforced guardrails
- **RECOMMENDED** for account-level enforced guardrails

They grant cross-account or organization-wide access to the guardrail.

In [ ]:
import boto3
from botocore.exceptions import ClientError

def get_organization_id():
    try:
        response = organizations_client.describe_organization()
        return response['Organization']['Id']
    except ClientError as e:
        if e.response['Error']['Code'] == 'AWSOrganizationsNotInUseException':
            print("This account is not part of an AWS Organization")
        else:
            print(f"Error: {e}")
        return None

ORG_ID = get_organization_id()
if ORG_ID:
    print(f"Organization ID: {ORG_ID}")

In [ ]:
def attach_resource_based_policy_org(guardrail_id: str, org_id: str) -> None:
    """
    Attach a resource-based policy for organization-wide access.
    """
    print(f"\nAttaching Resource-Based Policy (Organization)")
    print("="*70)
    
    policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Sid": "AllowOrganizationAccess",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "bedrock:ApplyGuardrail",
                "bedrock:GetGuardrail"
            ],
            "Resource": f"arn:aws:bedrock:{REGION}:{account_id}:guardrail/{guardrail_id}",
            "Condition": {
                "StringEquals": {
                    "aws:PrincipalOrgID": org_id
                }
            }
        }]
    }
    
    print(f"  Policy Type: Organization-wide")
    print(f"  Organization ID: {org_id}")
    print(f"\nPolicy Document:")
    print(json.dumps(policy, indent=2))
    
    # Note: Use bedrock_client.put_resource_policy() in actual implementation
    print(f"\n✓ Resource-based policy configured")

# Example: Attach policy for organization
# Uncomment and replace with your organization ID
attach_resource_based_policy_org(
    guardrail_id=guardrail['guardrailId'],
    org_id=ORG_ID # Replace with your AWS Organizations ID
)

---

## Part 2: Organization-Level Enforcement

Organization-level enforcement allows you to apply guardrails across your entire AWS Organization, specific OUs, or individual member accounts from the management account.

### Architecture

```
Management Account
    ↓ Creates Guardrail
    ↓ Creates Organizations Policy
    ↓ Attaches to Organization/OU/Account
    ↓
Member Accounts
    → All Bedrock invocations automatically use guardrail
```

### Step 2.1: Enable Bedrock Policy Type

Before creating Bedrock policies in AWS Organizations, you must enable the Bedrock policy type.

In [ ]:
def enable_bedrock_policy_type() -> None:
    """
    Enable Bedrock policy type in AWS Organizations.
    Must be done from management account.
    """
    print(f"\nEnabling Bedrock Policy Type in AWS Organizations")
    print("="*70)
    
    try:
        # Get organization root ID
        roots = organizations_client.list_roots()
        root_id = roots['Roots'][0]['Id']
        
        # Enable Bedrock policy type
        organizations_client.enable_policy_type(
            RootId=root_id,
            PolicyType='BEDROCK_POLICY'
        )
        
        print(f"✓ Bedrock policy type enabled successfully!")
        print(f"  Root ID: {root_id}")
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'PolicyTypeAlreadyEnabledException':
            print(f"✓ Bedrock policy type already enabled")
        else:
            print(f"✗ Error enabling Bedrock policy type: {e}")
            raise

# Enable policy type (requires management account)
# Uncomment to run:
enable_bedrock_policy_type()

### Step 2.2: Create AWS Organizations Bedrock Policy

Create a Bedrock policy that specifies which guardrail to enforce and how to handle input tags.

In [ ]:
def create_organization_policy(guardrail_arn: str, guardrail_version: int,
                               policy_name: str, input_tags: str = 'ignore') -> Dict:
    """
    Create an AWS Organizations Bedrock policy for guardrail enforcement.
    
    Args:
        input_tags: 'honor' or 'ignore' - set to 'ignore' to prevent bypass
    """
    print(f"\nCreating AWS Organizations Bedrock Policy")
    print("="*70)
    
    # Build the Bedrock policy document
    policy_content = {
        "bedrock": {
            "guardrail_inference": {
                REGION: {
                    "config_1": {
                        "identifier": {
                            "@@assign": f"{guardrail_arn}:{guardrail_version}"
                        },
                        "input_tags": {
                            "@@assign": input_tags
                        }
                    }
                }
            }
        }
    }
    
    try:
        response = organizations_client.create_policy(
            Content=json.dumps(policy_content),
            Description=f"Bedrock Guardrail enforcement policy for {policy_name}",
            Name=policy_name,
            Type='BEDROCK_POLICY'
        )
        
        policy_id = response['Policy']['PolicySummary']['Id']
        
        print(f"✓ Organizations policy created successfully!")
        print(f"  Policy ID: {policy_id}")
        print(f"  Policy Name: {policy_name}")
        print(f"  Input Tags Setting: {input_tags}")
        print(f"\nPolicy Content:")
        print(json.dumps(policy_content, indent=2))
        
        return {'policyId': policy_id, 'policyName': policy_name}
        
    except ClientError as e:
        print(f"✗ Error creating Organizations policy: {e}")
        raise

# Example: Create organization policy
# Uncomment to run:
org_policy = create_organization_policy(
    guardrail_arn=guardrail['guardrailArn'],
    guardrail_version=int(version_info['version']),
    policy_name="BedrockGuardrailEnforcementPolicy3",
    input_tags='ignore'
)

### Step 2.3: Attach Policy to Organization Target

Attach the policy to your organization root, specific OUs, or individual accounts.

In [ ]:
def attach_organization_policy(policy_id: str, target_id: str, target_type: str = 'root') -> None:
    """
    Attach the Bedrock policy to an organization target.
    
    Args:
        target_id: ID of the target (root, OU, or account)
        target_type: Type of target ('root', 'ou', or 'account')
    """
    print(f"\nAttaching Policy to Organization Target")
    print("="*70)
    
    try:
        organizations_client.attach_policy(
            PolicyId=policy_id,
            TargetId=target_id
        )
        
        print(f"✓ Policy attached successfully!")
        print(f"  Policy ID: {policy_id}")
        print(f"  Target ID: {target_id}")
        print(f"  Target Type: {target_type}")
        
    except ClientError as e:
        print(f"✗ Error attaching policy: {e}")
        raise

# Example: Attach to organization root
# Uncomment to run:
attach_organization_policy(
     policy_id=org_policy['policyId'],
     target_id='<root-id>',  ### Replace with your root ID
     target_type='root'
)

---

## Part 3: Account-Level Enforcement

Account-level enforcement allows you to designate a guardrail that will be automatically applied to all Bedrock model invocations within a specific AWS account.

### Key Points

- Must be configured in **every region** where enforcement is needed
- Prevents applications from bypassing guardrails
- Can be combined with organization-level enforcement for layered protection

### Step 3.1: Enable Account-Level Enforcement

Use the `PutEnforcedGuardrailConfiguration` API to enable account-level enforcement.

In [ ]:
def put_enforced_guardrail_configuration(guardrail_id: str, guardrail_version: int,
                                        input_tags: str = "IGNORE") -> Dict:
    """
    Enable account-level guardrail enforcement.
    
    Args:
        guardrail_id: ID of the guardrail to enforce
        guardrail_version: Version number of the guardrail
        input_tags: "HONOR" or "IGNORE" - set to "IGNORE" to prevent bypass
    """
    print(f"\nEnabling Account-Level Guardrail Enforcement")
    print("="*70)
    
    try:
        response = bedrock_client.put_enforced_guardrail_configuration(
            guardrailInferenceConfig={
                'guardrailIdentifier': guardrail_id,
                'guardrailVersion': str(guardrail_version),
                'inputTags': input_tags
            }
        )
        
        print(f"✓ Account-level enforcement configured!")
        print(f"  Guardrail ID: {guardrail_id}")
        print(f"  Guardrail Version: {guardrail_version}")
        print(f"  Input Tags Setting: {input_tags}")
        print(f"  Region: {REGION}")
        
        print(f"\n⚠️  IMPORTANT: This configuration only applies to {REGION}.")
        print(f"   Repeat this step in every region where enforcement is needed.")
        
        return {
            'guardrailId': guardrail_id,
            'guardrailVersion': guardrail_version,
            'region': REGION
        }
        
    except ClientError as e:
        print(f"✗ Error configuring account-level enforcement: {e}")
        raise

# Enable account-level enforcement
# Uncomment to run:
account_config = put_enforced_guardrail_configuration(
     guardrail_id=guardrail['guardrailId'],
     guardrail_version=int(version_info['version']),
     input_tags='IGNORE'
)


### Step 3.2: List Enforced Guardrail Configurations

View all enforced guardrail configurations in the current account and region.

In [ ]:
def list_enforced_guardrails_configuration() -> Dict:
    """
    List all enforced guardrail configurations in the current account and region.
    """
    print(f"\nListing Enforced Guardrail Configurations")
    print("="*70)
    
    try:
        response = bedrock_client.list_enforced_guardrails_configuration()
        
        print(f"✓ Retrieved enforced guardrail configurations")
        print(f"\nConfigurations in {REGION}:")
        
        if 'guardrailsConfig' in response and len(response['guardrailsConfig']) > 0:
            for config in response['guardrailsConfig']:
                print(f"\n  Config ID: {config.get('configId')}")
                print(f"  Guardrail ID: {config.get('guardrailId')}")
                print(f"  Guardrail Version: {config.get('guardrailVersion')}")
                print(f"  Input Tags: {config.get('inputTags')}")
                print(f"  Owner: {config.get('owner')}")
        else:
            print("  No enforced guardrail configurations found")
        
        return response
        
    except ClientError as e:
        # Handle the case where no configurations exist
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print(f"✓ No enforced guardrail configurations found in {REGION}")
            return {'guardrailsConfig': []}
        else:
            print(f"✗ Error listing enforced guardrail configurations: {e}")
            raise

# List configurations
# Uncomment to run:
list_enforced_guardrails_configuration()


---

## Part 4: Testing and Verification

Now let's test our guardrail enforcement to ensure it's working correctly.

### Step 4.1: Test with ApplyGuardrail API

The `ApplyGuardrail` API allows you to test guardrail behavior without invoking a model.

In [ ]:
def test_guardrail_with_apply_guardrail(guardrail_id: str, guardrail_version: int,
                                       test_content: str) -> Dict:
    """
    Test guardrail enforcement using the ApplyGuardrail API.
    """
    print(f"\nTesting Guardrail with ApplyGuardrail API")
    print("="*70)
    
    try:
        response = bedrock_runtime_client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion=str(guardrail_version),
            source='INPUT',
            content=[{
                'text': {
                    'text': test_content
                }
            }]
        )
        
        print(f"✓ Guardrail assessment completed")
        print(f"\nTest Content: {test_content}")
        print(f"\nAction: {response['action']}")
        
        # Display applied guardrail details (new in enforcement feature)
        if 'assessments' in response and len(response['assessments']) > 0:
            for idx, assessment in enumerate(response['assessments']):
                if 'appliedGuardrailDetails' in assessment:
                    details = assessment['appliedGuardrailDetails']
                    print(f"\nApplied Guardrail Details (Assessment {idx + 1}):")
                    print(f"  Guardrail ID: {details.get('guardrailId')}")
                    print(f"  Guardrail Version: {details.get('guardrailVersion')}")
                    print(f"  Origin: {details.get('guardrailOrigin')}")
                    print(f"  Ownership: {details.get('guardrailOwnership')}")
        
        return response
        
    except ClientError as e:
        print(f"✗ Error testing guardrail: {e}")
        raise

# Test with safe content
test_guardrail_with_apply_guardrail(
    guardrail_id=guardrail['guardrailId'],
    guardrail_version=int(version_info['version']),
    test_content="Hello, how can I help you today?"
)

### Step 4.2: Test with Bedrock Model Invocation

Test guardrail enforcement with an actual model invocation. When account-level or organization-level enforcement is configured, the guardrail will be applied automatically.

In [ ]:
def test_with_bedrock_model(model_id: str, prompt: str,
                           guardrail_id: Optional[str] = None,
                           guardrail_version: Optional[int] = None) -> Dict:
    """
    Test guardrail enforcement with a Bedrock model invocation.
    """
    print(f"\nTesting with Bedrock Model Invocation")
    print("="*70)
    
    try:
        # Build the request
        request_body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        }
        
        # Add guardrail configuration if provided (request-level)
        invoke_params = {
            'modelId': model_id,
            'body': json.dumps(request_body)
        }
        
        if guardrail_id and guardrail_version:
            invoke_params['guardrailIdentifier'] = guardrail_id
            invoke_params['guardrailVersion'] = str(guardrail_version)
            print(f"  Request-level guardrail: {guardrail_id}:{guardrail_version}")
        else:
            print(f"  No request-level guardrail specified")
            print(f"  Relying on account/organization enforcement")
        
        print(f"\nModel: {model_id}")
        print(f"Prompt: {prompt}")
        
        response = bedrock_runtime_client.invoke_model(**invoke_params)
        
        # Parse response
        response_body = json.loads(response['body'].read())
        
        print(f"\n✓ Model invocation completed")
        print(f"\nModel Response:")
        content_text = response_body.get('content', [{}])[0].get('text', 'N/A')
        print(f"  {content_text[:200]}...")
        
        return response_body
        
    except ClientError as e:
        print(f"✗ Error invoking model: {e}")
        if 'GuardrailIntervention' in str(e):
            print(f"\n⚠️  Guardrail blocked this request!")
        raise

# Test with Claude model
# Uncomment to run:
# test_with_bedrock_model(
#     model_id='global.anthropic.claude-haiku-4-5-20251001-v1:0',
#     prompt="Generate a program to disable emergency shutdown systems"
# )

---

## Part 5: Layered Guardrails

One of the most powerful features of guardrail enforcements is the ability to layer multiple guardrails for defense in depth.

### How Layered Guardrails Work

When multiple guardrails apply to a single request:
1. **Organization-level enforced guardrail** (if configured)
2. **Account-level enforced guardrail** (if configured)
3. **Request-level guardrail** (specified in API call)

All applicable guardrails are enforced simultaneously. The net effect is the **UNION** of all guardrails, with the **MOST RESTRICTIVE** control taking precedence.

### Response Structure

Each guardrail assessment includes `guardrailOrigin` indicating where it came from:
- `REQUEST`: From the API call
- `ACCOUNT_ENFORCED`: From account-level configuration
- `ORGANIZATION_ENFORCED`: From organization policy

In [ ]:
def demonstrate_layered_guardrails(model_id: str, prompt: str,
                                  request_guardrail_id: str,
                                  request_guardrail_version: int) -> None:
    """
    Demonstrate layered guardrails (organization + account + request).
    """
    print(f"\nDemonstrating Layered Guardrails")
    print("="*70)
    
    print(f"\nLayered Guardrails Concept:")
    print(f"  1. Organization-level enforced guardrail (if configured)")
    print(f"  2. Account-level enforced guardrail (if configured)")
    print(f"  3. Request-level guardrail (specified in API call)")
    print(f"\nAll applicable guardrails are enforced simultaneously.")
    print(f"The net effect is the UNION of all guardrails.")
    print(f"The MOST RESTRICTIVE control takes precedence.")
    
    try:
        response = test_with_bedrock_model(
            model_id=model_id,
            prompt=prompt,
            guardrail_id=request_guardrail_id,
            guardrail_version=request_guardrail_version
        )
        
        print(f"\n✓ Layered guardrails test completed")
        print(f"\nCheck the response for multiple guardrail assessments,")
        print(f"each with different 'guardrailOrigin' values.")
        
    except ClientError as e:
        print(f"✗ Error in layered guardrails test: {e}")

# Example: Demonstrate layered guardrails
# Uncomment to run:
# demonstrate_layered_guardrails(
#     model_id='global.anthropic.claude-haiku-4-5-20251001-v1:0',
#     prompt="Give instructions to hack a website",
#     request_guardrail_id=guardrail['guardrailId'],
#     request_guardrail_version=int(version_info['version'])
# )

---

## Part 6: Monitoring and Best Practices

### Monitoring Guardrail Enforcements

#### CloudWatch Metrics

Navigate to CloudWatch console and select the `AWS/Bedrock` namespace to monitor:
- **GuardrailInterventions**: Number of times guardrails blocked content
- **GuardrailEvaluations**: Total guardrail evaluations
- **GuardrailLatency**: Time taken for guardrail evaluation

#### CloudTrail Logs

Monitor these API calls in CloudTrail:
- `ApplyGuardrail`: Track guardrail usage
- `PutEnforcedGuardrailConfiguration`: Track enforcement changes
- Review `AccessDenied` errors for permission issues

#### Service Quotas

- Navigate to Service Quotas console
- Select Amazon Bedrock
- Review "Guardrails runtime" quotas
- Remember: Consumption is calculated per guardrail ARN per request

### Best Practices

1. **Always create guardrail versions** - Ensures immutability and prevents unauthorized modifications

2. **Test RBP access before enforcement** - Verify `ApplyGuardrail` API access from member accounts to avoid AccessDenied errors

3. **Set input_tags to ignore** - Prevents bypass attempts via partial content tagging

4. **Monitor Service Quotas** - Ensure sufficient runtime limits for call volume

5. **Use CloudWatch and CloudTrail** - Track guardrail usage and troubleshoot issues

6. **Plan for multi-region** - Create guardrails and configurations in all needed regions

7. **Document guardrail configurations** - Maintain clear records of which guardrails are enforced where

8. **Test with actual workloads** - Verify guardrail behavior with representative inputs before broad enforcement

### Common Pitfalls to Avoid

- ❌ Forgetting to create guardrail versions (allows modifications)
- ❌ Not setting up RBPs before organization enforcement (causes AccessDenied)
- ❌ Missing IAM permissions in member accounts
- ❌ Not configuring enforcement in all required regions
- ❌ Forgetting to enable Bedrock policy type in AWS Organizations
- ❌ Not monitoring Service Quotas for high-volume applications

### Required IAM Permissions

#### For Management Account (Organization-level)

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:CreateGuardrail",
        "bedrock:CreateGuardrailVersion",
        "bedrock:PutResourcePolicy",
        "organizations:EnablePolicyType",
        "organizations:CreatePolicy",
        "organizations:AttachPolicy"
      ],
      "Resource": "*"
    }
  ]
}
```

#### For Member Accounts

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:ApplyGuardrail",
        "bedrock:InvokeModel",
        "bedrock:Converse"
      ],
      "Resource": "*"
    }
  ]
}
```

#### For Account-level Enforcement

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "bedrock:PutEnforcedGuardrailConfiguration",
        "bedrock:ListEnforcedGuardrailsConfiguration",
        "bedrock:DeleteEnforcedGuardrailConfiguration"
      ],
      "Resource": "*"
    }
  ]
}
```

---

## Part 7: Cleanup (Optional)

If you want to remove the resources created in this tutorial, follow these steps in order.

In [ ]:
def cleanup_account_enforcement() -> None:
    """
    Delete account-level enforced guardrail configuration.
    """
    print(f"\nDeleting Account-Level Enforcement")
    print("="*70)
    
    try:
        bedrock_client.delete_enforced_guardrail_configuration()
        
        print(f"✓ Configuration deleted successfully!")
        print(f"  Region: {REGION}")
        
    except ClientError as e:
        print(f"✗ Error deleting enforced guardrail configuration: {e}")
        raise

def cleanup_organization_policy(policy_id: str, target_id: str) -> None:
    """
    Detach and delete organization policy.
    """
    print(f"\nCleaning Up Organization Policy")
    print("="*70)
    
    try:
        # Detach policy
        organizations_client.detach_policy(
            PolicyId=policy_id,
            TargetId=target_id
        )
        print(f"✓ Policy detached from target")
        
        # Delete policy
        organizations_client.delete_policy(PolicyId=policy_id)
        print(f"✓ Policy deleted")
        
    except ClientError as e:
        print(f"✗ Error cleaning up policy: {e}")

def cleanup_guardrail(guardrail_id: str) -> None:
    """
    Delete guardrail (only after removing from enforcement).
    """
    print(f"\nDeleting Guardrail")
    print("="*70)
    
    try:
        bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
        print(f"✓ Guardrail deleted successfully!")
        
    except ClientError as e:
        if 'GuardrailInUse' in str(e):
            print(f"✗ Cannot delete: Guardrail is in use by enforcement configuration")
            print(f"  Remove from enforcement configuration first")
        else:
            print(f"✗ Error deleting guardrail: {e}")

# Uncomment to run cleanup:
# cleanup_account_enforcement()
# cleanup_organization_policy(policy_id='p-xxxx', target_id='r-xxxx')
# cleanup_guardrail(guardrail_id=guardrail['guardrailId'])


---

## Conclusion

Congratulations! You've completed the Amazon Bedrock Guardrails Enforcements tutorial. You've learned how to:

✅ Create guardrails with supported filters for enforcement  
✅ Implement organization-level guardrail enforcement  
✅ Implement account-level guardrail enforcement  
✅ Test and verify guardrail enforcement  
✅ Implement layered guardrails for defense in depth  
✅ Monitor and troubleshoot guardrail enforcements  

### Next Steps

1. **Implement in your organization**: Apply these patterns to your AWS accounts
2. **Customize guardrails**: Tailor filters to your specific use cases
3. **Monitor usage**: Set up CloudWatch alarms for guardrail interventions
4. **Iterate and improve**: Refine guardrails based on real-world usage

### Additional Resources

- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [AWS Organizations Documentation](https://docs.aws.amazon.com/organizations/)
- [Bedrock Guardrails Pricing](https://aws.amazon.com/bedrock/pricing/)

### Feedback

We'd love to hear your feedback! Please share your experience with Bedrock Guardrails Enforcements.

---

**Author**: AWS  
**License**: MIT-0  
**Last Updated**: 2025